## All Import Modules

In [1]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd

## MENUPLIATE INPUT DATA

In [2]:
previes_days = 10
def Main_Dataframe(symbol,previes_days):
    # ------------------------------------ DAY TIME FRAME -----------------------------------------------------------
    ticker = yf.Ticker(symbol)
    day_df = ticker.history(period=f"80d", interval="1d")
    
    # Reset the index and convert the 'Date' column to date format
    day_df.reset_index(inplace=True)
    day_df['Date'] = day_df['Date'].dt.date
    
    # Calculate technical indicators
    day_df['Day_10EMA'] = trend.EMAIndicator(day_df['Close'], window=10).ema_indicator()
    
    day_df['Day_RSI'] = momentum.RSIIndicator(day_df['Close'], window=3).rsi()
    day_df['Day_RSI_EMA'] = trend.EMAIndicator(day_df['Day_RSI'], window=6).ema_indicator()

    # -----------ADD EXTRA INDICATORS HERE --------------
    day_df['Day_Trend'] = np.select(
    [day_df['Day_10EMA'] > day_df['Day_10EMA'].shift(1),
     day_df['Day_10EMA'] < day_df['Day_10EMA'].shift(1)],
    ['Uptrend', 'Downtrend'],'Sideways')
    # Rename columns and drop unnecessary columns
    day_df.rename(columns={'Open': 'Day_Open', 'High': 'Day_High', 'Low': 'Day_Low', 'Close': 'Day_Close'}, inplace=True)
    day_df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Day_10EMA','Day_RSI'], inplace=True)
    day_df.dropna(inplace=True)

    # ------------------------------------ 5 MIN TIME FRAME -----------------------------------------------------------
    # previes_days = 5
    df = ticker.history(period=f"{previes_days}d", interval="5m")
    df.reset_index(inplace=True)
    df['Date'] = df['Datetime'].dt.date
    df['Time'] = df['Datetime'].dt.time
    
    df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Datetime'], inplace=True)
    df.dropna(inplace=True)
    # -----------ADD EXTRA INDICATORS HERE --------------
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=5).ema_indicator()
    df['15EMA'] = trend.EMAIndicator(df['Close'], window=15).ema_indicator()
    df['Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    df['Prev_5EMA'] = df['5EMA'].shift(1)
    df['Prev_15EMA'] = df['15EMA'].shift(1)

    df['RSI'] = momentum.RSIIndicator(df['Close'], window=3).rsi()
    df['RSI_EMA'] = trend.EMAIndicator(df['RSI'], window=6).ema_indicator()
    
   
    short_ema =trend.EMAIndicator(df["Close"], window=5)
    long_ema = trend.EMAIndicator(df["Close"], window=15)
    
    df["5EMA"] = short_ema.ema_indicator()
    df["15EMA"] = long_ema.ema_indicator()
    
    # Determine the perfect trend based on EMA crossovers
    df["Trend"] = "Sideways"  # Default to Sideways
    df.loc[(df["5EMA"] > df["15EMA"]) & (df["5EMA"].shift(1) <= df["15EMA"].shift(1)), "Trend"] = "Uptrend"
    df.loc[(df["5EMA"] < df["15EMA"]) & (df["5EMA"].shift(1) >= df["15EMA"].shift(1)), "Trend"] = "Downtrend"

    first_candle_high = df.groupby('Date')['High'].first()
    first_candle_low = df.groupby('Date')['Low'].first()
    
    # Create a new column for the first candle's high and low
    df['First_Candle_High'] = df['Date'].map(first_candle_high)
    df['First_Candle_Low'] = df['Date'].map(first_candle_low)

    # ------------------------------------ MERGE DATA FRAME -----------------------------------------------------------
    
    merged_df = day_df.merge(df, on='Date', how='inner')
    merged_df.dropna(inplace=True)
    merged_df['DateTime'] = pd.to_datetime(merged_df['Date'].astype(str) + ' ' + merged_df['Time'].astype(str))
    merged_df.drop(['Date', 'Time'], axis=1, inplace=True)
    return round(merged_df,2)
    # return day_df
    # return df

data = Main_Dataframe("^NSEI",previes_days)
# data = Main_Dataframe("HDFCBANK.BO")
# data.tail(5)

In [3]:
all_columsn = ['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA',
        'Open', 'High', 'Low', 'Close', '5EMA', '15EMA',
       'Prev_5EMA', 'Prev_15EMA', 'RSI', 'RSI_EMA', 'First_Candle_High', 'First_Candle_Low']
day_all_columsn = ['Date', 'Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA']
# data.iloc[-2]

# Add Signals Columns

In [4]:
def Add_Signals(dfnew):
    df = dfnew.copy()
    df['Signal_Day_RSI'] = np.select([df['Day_RSI_EMA'] <= 30, df['Day_RSI_EMA'] >= 70],['CE', 'PE'],default='0')
    
    # df['Signal_RSI'] = np.select([df['RSI_EMA'] <= 70, df['RSI_EMA'] >= 30],['CE', 'PE'],default='0')
    df['Signal_RSI_CE'] = np.where(df['RSI_EMA'] <= 50, 'CE', '0')
    df['Signal_RSI_PE'] = np.where(df['RSI_EMA'] >= 50, 'PE', '0')


    ce_5ema = (df['High'].shift(1) < df['5EMA'].shift(1)) & (df['Close'] >= df['5EMA'])
    pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) & (df['Close'] <= df['5EMA'])
    
    df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='NONE')

    ce_ema_crossover = (df['5EMA'] > df['15EMA']) & (df['5EMA'].shift(1) <= df['15EMA'].shift(1))
    pe_ema_crossover = (df['5EMA'] < df['15EMA']) & (df['5EMA'].shift(1) >= df['15EMA'].shift(1))
    
    df['Signal_EMA_CROSSOVER'] = np.select([ce_ema_crossover, pe_ema_crossover],['CE', 'PE'], default='0')
    
    # df.drop(columns = all_columsn,inplace= True)
    return df
newdata = Add_Signals(data)
# newdata

In [5]:
# newdata[(newdata['Signal_RSI'] == 'PE') & (newdata['Signal_EMA_CROSSOVER'] == 'PE')]
# newdata[(newdata['Signal_EMA_CROSSOVER'] == 'PE')].shape[0]
# newdata[(newdata['Signal_RSI_PE'] == 'PE')].shape[0]

# ENTRY EXIT LOGIC

In [6]:
def CE_ENTRY(row):
    status =  (row['Signal_EMA_CROSSOVER'] == "CE") and (row['Signal_RSI_CE'] == "CE")
    return status
    
def CE_EXIT(row):
    status =  row['Signal_EMA_CROSSOVER'] == "PE" 
    return status
    
def PE_ENTRY(row):
    status =  (row['Signal_EMA_CROSSOVER'] == "PE") and (row['Signal_RSI_PE'] == "PE")
    return status
    
def PE_EXIT(row):
    status =  row['Signal_EMA_CROSSOVER'] == "CE" 
    return status
    

# VARIABLES 

In [7]:
CE_HOLDING = False
PE_HOLDING = False
CE_data = pd.DataFrame(columns=['Entry Position', 'Entry DateTime', 'Exit DateTime', 'Current P&L', 'Exit Position','Symbol'])
PE_data = pd.DataFrame(columns=['Entry Position', 'Entry DateTime', 'Exit DateTime', 'Current P&L', 'Exit Position','Symbol'])
RISK = 30
TARGET = 70
def Main_Strategy_Run(row):
    
    global CE_data
    global PE_data
    global CE_HOLDING
    global PE_HOLDING

    if CE_ENTRY(row) and CE_HOLDING == False:
        entry_price = row['Close']
        ce_qnty = 1
        ce_stoploss = entry_price - RISK
        ce_target = entry_price + TARGET
        entry_time = row['DateTime']  
        CE_HOLDING = True

        ce_entry_data = {
                'Entry Position': entry_price,
                'Entry DateTime': entry_time,
                'Stoploss' : ce_stoploss,
                'Target' : ce_target,
                'Exit DateTime': None,
                'Current P&L': None,
                'Exit Position': None,
                'Symbol': "CE",
                'Exit Status': None,
                'Duration': None
            }
        
        CE_data = pd.concat([CE_data, pd.DataFrame([ce_entry_data])], ignore_index=True)
    

    if CE_HOLDING and ((row['Close'] <= CE_data.iloc[-1]['Stoploss']) or (row['Close'] >= CE_data.iloc[-1]['Target'])):
        exit_price = row['Close']
        if exit_price <= CE_data.iloc[-1]['Stoploss']:
            exit_status = "Stoploss"
        elif exit_price >= CE_data.iloc[-1]['Target']:
            exit_status = "Target"
            
        CE_HOLDING = False
        CE_data.loc[CE_data.index[-1], 'Exit DateTime'] = row['DateTime']
        CE_data.loc[CE_data.index[-1], 'Exit Position'] = row['Close']
        CE_data.loc[CE_data.index[-1], 'Current P&L'] = exit_price - CE_data.iloc[-1]['Entry Position']
        CE_data.loc[CE_data.index[-1], 'Exit Status'] = exit_status
        CE_data.loc[CE_data.index[-1], 'Duration'] = (row['DateTime'] - CE_data.iloc[-1]['Entry DateTime']).total_seconds() / 60

        


    if PE_ENTRY(row) and PE_HOLDING == False:
        entry_price = row['Close']
        pe_qnty = 1
        pe_stoploss = entry_price + RISK
        pe_target = entry_price - TARGET
        entry_time = row['DateTime']  
        PE_HOLDING = True

        pe_entry_data = {
                'Entry Position': entry_price,
                'Entry DateTime': entry_time,
                'Stoploss' : pe_stoploss,
                'Target' : pe_target,
                'Exit DateTime': None,
                'Current P&L': None,
                'Exit Position': None,
                'Symbol': "PE",
                'Exit Status': None,
                'Duration': None
            }
        
        PE_data = pd.concat([PE_data, pd.DataFrame([pe_entry_data])], ignore_index=True)

    if PE_HOLDING and ((row['Close'] >= PE_data.iloc[-1]['Stoploss']) or (row['Close'] <= PE_data.iloc[-1]['Target'])):
        exit_price = row['Close']
        if exit_price >= PE_data.iloc[-1]['Stoploss']:
            exit_status = "Stoploss"
        elif exit_price <= PE_data.iloc[-1]['Target']:
            exit_status = "Target"
            
        PE_HOLDING = False
        PE_data.loc[PE_data.index[-1], 'Exit DateTime'] = row['DateTime']
        PE_data.loc[PE_data.index[-1], 'Exit Position'] = row['Close']
        PE_data.loc[PE_data.index[-1], 'Current P&L'] = PE_data.iloc[-1]['Entry Position']- exit_price
        PE_data.loc[PE_data.index[-1], 'Exit Status'] = exit_status
        PE_data.loc[PE_data.index[-1], 'Duration'] = (row['DateTime'] - PE_data.iloc[-1]['Entry DateTime']).total_seconds() / 60


newdata.apply(Main_Strategy_Run, axis=1)
Tred_Log = pd.concat([CE_data, PE_data]).sort_values('Entry DateTime').reset_index(drop=True)
Tred_Log

,Entry Position,Entry DateTime,Exit DateTime,Current P&L,Exit Position,Symbol,Stoploss,Target,Exit Status,Duration
0,19369.70,2023-11-10 11:20:00,2023-11-10 15:25:00,73.75,19443.45,CE,19339.70,19439.70,Target,245.0
1,19441.85,2023-11-13 13:20:00,2023-11-15 09:15:00,171.8,19613.65,CE,19411.85,19511.85,Target,2635.0
2,19666.65,2023-11-15 15:00:00,2023-11-16 09:15:00,-31.2,19635.45,CE,19636.65,19736.65,Stoploss,1095.0
3,19635.45,2023-11-16 09:15:00,2023-11-16 09:35:00,-47.15,19682.6,PE,19665.45,19565.45,Stoploss,20.0
4,19763.60,2023-11-17 12:45:00,2023-11-20 11:10:00,85.6,19678.0,PE,19793.60,19693.60,Target,4225.0
5,19685.05,2023-11-20 12:55:00,2023-11-20 14:45:00,-33.25,19718.3,PE,19715.05,19615.05,Stoploss,110.0
6,19705.45,2023-11-20 14:20:00,2023-11-21 09:25:00,71.9,19777.35,CE,19675.45,19775.45,Target,1145.0
7,19744.00,2023-11-22 13:40:00,2023-11-22 14:25:00,-33.65,19777.65,PE,19774.00,19674.00,Stoploss,45.0
8,19797.45,2023-11-24 15:15:00,None,None,None,CE,19767.45,19867.45,None,None


In [8]:
positive_trades = Tred_Log[Tred_Log['Current P&L'] > 0]
negative_trades = Tred_Log[Tred_Log['Current P&L'] < 0]
accuracy = len(positive_trades) / len(Tred_Log) * 100

In [9]:
positive_trades = Tred_Log[Tred_Log['Current P&L'] > 0]
negative_trades = Tred_Log[Tred_Log['Current P&L'] < 0]
accuracy = len(positive_trades) / len(Tred_Log) * 100
ce_trades = Tred_Log[Tred_Log['Symbol'] == 'CE']
pe_trades = Tred_Log[Tred_Log['Symbol'] == 'PE']
ce_Positive = ce_trades[ce_trades['Current P&L'] > 0].shape[0]
ce_Nagetive = ce_trades[ce_trades['Current P&L'] < 0].shape[0]
pe_Positive = pe_trades[pe_trades['Current P&L'] > 0].shape[0]
pe_Nagetive = pe_trades[pe_trades['Current P&L'] < 0].shape[0]
total_pnl = round(Tred_Log['Current P&L'].sum(),2)
print("Total CE  Tred :",ce_trades.shape[0] ,"|| Positive :",ce_Positive ,"| Nagative :",ce_Nagetive)
print("Total PE  Tred :",pe_trades.shape[0] ,"|| Positive :",pe_Positive ,"| Nagative :",pe_Nagetive)
print("Accuracy: {:.2f}%".format(accuracy))
print("Total Tred :",Tred_Log.shape[0])
print("Total Positive Tred :",positive_trades.shape[0])
print("Total Nagative Tred :",negative_trades.shape[0])
print("TOTAL Pr:", total_pnl ,"in" ,previes_days ,"Day")
print("RISK TARGET RATIO :",RISK,"/",TARGET)

Total CE  Tred : 5 || Positive : 3 | Nagative : 1
Total PE  Tred : 4 || Positive : 1 | Nagative : 3
Accuracy: 44.44%
Total Tred : 9
Total Positive Tred : 4
Total Nagative Tred : 4
TOTAL Pr: 257.8 in 10 Day
RISK TARGET RATIO : 30 / 70
